# 2020 US Election Progress Analysis

https://www.theguardian.com/us-news/ng-interactive/2020/nov/07/us-election-2020-live-results-donald-trump-joe-biden-presidential-votes-pennsylvania-georgia-arizona-nevada

In [ ]:
import requests
import pandas as pd
import datetime
import dateutil.parser
import json
import re
import glob
import sys
import locale
locale.setlocale(locale.LC_ALL, '')  # Use '' for auto, or force e.g. to 'en_US.UTF-8'

# gimme 100% width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

def format_percent(x):
    return f'{x*100:.2f}'

def flattened(l):
    return [item for sublist in l for item in sublist]

NAME_BY_ACRONYM = {
    'AL': 'Alabama',        'AK': 'Alaska',        'AZ': 'Arizona',    'AR': 'Arkansas',             'CA': 'California',
    'CO': 'Colorado',       'CT': 'Connecticut',   'DE': 'Delaware',   'DC': 'District of Columbia', 'FL': 'Florida',
    'GA': 'Georgia',        'HI': 'Hawaii',        'ID': 'Idaho',      'IL': 'Illinois',             'IN': 'Indiana',
    'IA': 'Iowa',           'KS': 'Kansas',        'KY': 'Kentucky',   'LA': 'Louisiana',            'ME': 'Maine',
    'MD': 'Maryland',       'MA': 'Massachusetts', 'MI': 'Michigan',   'MN': 'Minnesota',            'MS': 'Mississippi',
    'MO': 'Missouri',       'MT': 'Montana',       'NE': 'Nebraska',   'NV': 'Nevada',               'NH': 'New Hampshire',
    'NJ': 'New Jersey',     'NM': 'New Mexico',    'NY': 'New York',   'NC': 'North Carolina',       'ND': 'North Dakota',
    'OH': 'Ohio',           'OK': 'Oklahoma',      'OR': 'Oregon',     'PA': 'Pennsylvania',         'RI': 'Rhode Island',
    'SC': 'South Carolina', 'SD': 'South Dakota',  'TN': 'Tennessee',  'TX': 'Texas',    'UT':       'Utah',
    'VT': 'Vermont',        'VA': 'Virginia',      'WA': 'Washington', 'WV': 'West Virginia',        'WI': 'Wisconsin',
    'WY': 'Wyoming',        'PR': 'Puerto Rico',
}

def load(file_name=None):
    if file_name:
        with open(file_name) as f:
            return json.load(f)
    
    # Load dataset
    try:
        # From the Guardian on the Web...
        last_update = requests.get('https://interactive.guim.co.uk/2020/11/us-general-election-data/prod/last_updated.json').json()
        last_time = last_update['time']
        url = f'https://interactive.guim.co.uk/2020/11/us-general-election-data/prod/data-out/{last_time}/president_details.json'
        print(last_time)
        data = requests.get(url).json()
        fname = 'data-cache--' + re.compile('[^0-9a-zA-Z-]').sub('.', last_time) + '.json'
        with open(fname, 'w') as f: json.dump(data, f, indent='  ', sort_keys=True)
    except Exception as e:
        # ... or cached
        print(e, file=sys.stderr)
        names = sorted(glob.glob('data-cache--*.json'))
        print('Found cache:', names)
        name = names[-1]
        print('Loading:', name)
        with open(name) as f: data = json.load(f)
    return data

def compute(data):
    electors_by_candidate = {}
    electors_by_candidate_may = {}
    electors_by_state = {}

    new_data = {}
    new_data_short = {}

    n = 0

    # Compute electors by state
    for acronym, state_data in data.items():
        if acronym == 'US': continue
        electors = max( c['electWon'] for c in state_data['candidates'])
        # Some return zero, fix that
        electors_by_state[acronym] = {'AK': 6, 'GA': 16}.get(acronym, electors)

    # Analyze each state
    for acronym, state_data in data.items():
        if acronym == 'US': continue
        n += 1
        extra = {}
        extra_short = {}
        electors = electors_by_state[acronym]

        # Analyze each field
        for k, v in state_data.items():
            if k == 'declarationTime' and v:
                # make it shorter
                dt = dateutil.parser.parse(v)
                state_data[k] = datetime.datetime.strftime(dt, '%b %d %H:%M')
            if k == 'candidates':
                candidate = {c['name'].split()[-1]:c for c in v}
                if n and n < 4:
                    pass #print();print(candidate)
                # compute votes
                nb_counted = sum([c['votes'] for c in v])
                max_votes = int(state_data['totalVotes'] / state_data['reporting'] * 100.)
                rem_votes = max_votes - nb_counted
                # general fields
                #extra['electors'] = electors
                extra['% counted']   = extra_short['% counted']   = state_data['reporting']
                extra['grand total'] = extra_short['grand total'] = max_votes
                extra['uncounted']   = extra_short['uncounted']   = rem_votes
                # per-candidate votes
                extra['- SCORE -'] = ''
                for name, cdata in candidate.items():
                    val = cdata['votes'] / nb_counted
                    extra[name]       = extra_short[name]       = cdata['votes']
                for name, cdata in candidate.items():
                    val = cdata['votes'] / nb_counted
                    extra[name + '%']       = format_percent(val)
                    extra_short[name + '%'] = val
                # winner
                winner = sorted([(cdata['votes'], name) for name, cdata in candidate.items()])[-1][1]
                extra['winner'] = winner
                # compute "maybe" winner: who could win, if getting all remaining votes
                extra['- MAYBE -'] = ''
                cand2 = {c['votes']:c['name'].split()[-1] for c in v}
                possibilities = [
                    [[votes2 + (rem_votes if name2!=winner else 0), name2] for votes2, name2 in cand2.items()]
                    for votes1, name1 in cand2.items()]
                winner_may_votes, winner_may = sorted(flattened(possibilities))[-1]
                for name, cdata in candidate.items():
                    val = (cdata['votes']+rem_votes) / (nb_counted+rem_votes)
                    extra[name + '%may'] = format_percent(val) if name != winner else f'({format_percent(val)})'
                extra['may'] = winner_may if winner_may != winner else ''
                #extra['may votes'] = winner_may_votes
                #extra['may %'] = format_percent(winner_may_votes / max_votes)

        # accumulate won electors
        electors_by_candidate.setdefault(winner, 0)
        electors_by_candidate[winner] += electors
        electors_by_candidate_may.setdefault(winner_may, 0)
        electors_by_candidate_may[winner_may] += electors

        # format ints
        for k, v in extra.items():
            if v.__class__ is int:
                extra[k] = f'{v:n}'
        # keep new data and format ints
        new_data[acronym] = extra
        new_data_short[acronym] = extra_short

    electors_by_candidate = [{'Candidate': cand, 'Electors': electors} for cand, electors in electors_by_candidate.items()]
    electors_by_candidate_may = [{'Candidate maybe': cand, 'Electors': electors} for cand, electors in electors_by_candidate_may.items()]
    electors_by_state = {acronym:{'Name': NAME_BY_ACRONYM[acronym], 'Electors': nb} for acronym, nb in electors_by_state.items()}
    
    return new_data, new_data_short, electors_by_candidate, electors_by_candidate_may, electors_by_state

def dic_dict_sub(d1, d2):
    def fmt(v):
        if v.__class__ == int:
            return f'{v:n}'
        elif v.__class__ == float:
            return f'{v*100:.2f}'
        else:
            return v
        
    return { k1:{k2:fmt(v2-d2[k1][k2]) for k2, v2 in v1.items()} for k1, v1 in d1.items()}

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 10000)
pd.set_option('display.max_colwidth', None)

# 1. Results as of Sat 7 Nov

In [ ]:
data = load('data-cache--2020-11-07T17.14.12.533.00.00.json')

new_data, new_data_short_0, electors_by_candidate, electors_by_candidate_may, electors_by_state = compute(data)
pd.DataFrame(new_data)

In [ ]:
pd.DataFrame(electors_by_state)

## Electors

In [ ]:
pd.DataFrame(electors_by_candidate)

## Potential twist 

In [ ]:
pd.DataFrame(electors_by_candidate_may)

# 2. Results as of Thu 12 Nov

In [ ]:
#data = load('data-cache--2020-11-09T23.58.18.360.00.00.json')
#data = load('data-cache--2020-11-11T07.57.21.383.00.00.json')
#data = load('data-cache--2020-11-12T12.27.02.803.00.00.json')
data = load('data-cache--2020-11-13T09.09.56.007.00.00.json')
new_data, new_data_short_1, electors_by_candidate, electors_by_candidate_may, electors_by_state = compute(data)
pd.DataFrame(new_data)

In [ ]:
pd.DataFrame(electors_by_state)

## Electors

In [ ]:
pd.DataFrame(electors_by_candidate)

## Potential twist 

In [ ]:
pd.DataFrame(electors_by_candidate_may)

# 3. Results as of now

In [ ]:
data = load()
new_data, new_data_short_2, electors_by_candidate, electors_by_candidate_may, electors_by_state = compute(data)
pd.DataFrame(new_data)

In [ ]:
pd.DataFrame(electors_by_state)

## Progression 

In [ ]:
pd.DataFrame(dic_dict_sub(new_data_short_2, new_data_short_1))

## Progression since Sat 7 Nov

In [ ]:
pd.DataFrame(dic_dict_sub(new_data_short_2, new_data_short_0))

## Electors

In [ ]:
pd.DataFrame(electors_by_candidate)

## Potential twist

In [ ]:
pd.DataFrame(electors_by_candidate_may)

# 4. Electors per state

In [ ]:
pd.DataFrame(electors_by_state).transpose()